In [1]:
#  Appliances Energy Prediction data. (Quiz Data)
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


qdat = pd.read_csv('energydata_complete.csv')
qcol_names = {'T1': 'Temperature in kitchen area','RH_1': 'Humidity in kitchen area','T2': 'Temperature in living room area',
    'RH_2': 'Humidity in living room area','T3': 'Temperature in laundry room area','RH_3': 'Humidity in laundry room area',
    'T4': 'Temperature in office room','RH_4': 'Humidity in office room','T5': 'Temperature in bathroom',
    'RH_5': 'Humidity in bathroom','T6': 'Temperature outside the building (north side)',
    'RH_6': 'Humidity outside the building (north side)','T7': 'Temperature in ironing room','RH_7': 'Humidity in ironing room',
    'T8': 'Temperature in teenager room 2','RH_8': 'Humidity in teenager room 2','T9': 'Temperature in parents room',
    'RH_9': 'Humidity in parents room','T_out': 'Temperature outside (from Chievres weather station)',
    'RH_out': 'Humidity outside (from Chievres weather station)','rv1': 'Random variable 1','rv2': 'Random variable 2',}

qdat = qdat.rename(columns = qcol_names)
qdat = qdat.drop(columns = ['date', 'lights'])
qdat

,Appliances,Temperature in kitchen area,Humidity in kitchen area,Temperature in living room area,Humidity in living room area,Temperature in laundry room area,Humidity in laundry room area,Temperature in office room,Humidity in office room,Temperature in bathroom,...,Temperature in parents room,Humidity in parents room,Temperature outside (from Chievres weather station),Press_mm_hg,Humidity outside (from Chievres weather station),Windspeed,Visibility,Tdewpoint,Random variable 1,Random variable 2
0,60,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,17.166667,...,17.033333,45.5300,6.600000,733.5,92.000000,7.000000,63.000000,5.300000,13.275433,13.275433
1,60,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,17.166667,...,17.066667,45.5600,6.483333,733.6,92.000000,6.666667,59.166667,5.200000,18.606195,18.606195
2,50,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,17.166667,...,17.000000,45.5000,6.366667,733.7,92.000000,6.333333,55.333333,5.100000,28.642668,28.642668
3,50,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,17.166667,...,17.000000,45.4000,6.250000,733.8,92.000000,6.000000,51.500000,5.000000,45.410389,45.410389
4,60,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,17.200000,...,17.000000,45.4000,6.133333,733.9,92.000000,5.666667,47.666667,4.900000,10.084097,10.084097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,100,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,45.590000,23.200000,...,23.200000,46.7900,22.733333,755.2,55.666667,3.333333,23.666667,13.333333,43.096812,43.096812
19731,90,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,45.590000,23.230000,...,23.200000,46.7900,22.600000,755.2,56.000000,3.500000,24.500000,13.300000,49.282940,49.282940
19732,270,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,45.730000,23.230000,...,23.200000,46.7900,22.466667,755.2,56.333333,3.666667,25.333333,13.266667,29.199117,29.199117
19733,420,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,45.790000,23.200000,...,23.200000,46.8175,22.333333,755.2,56.666667,3.833333,26.166667,13.233333,6.322784,6.322784


In [2]:
# CHECKING ACCURACY OF THE MODEL for QDAT

#Firstly, we normalise our dataset to a common scale using the min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_qdat = pd.DataFrame(scaler.fit_transform(qdat), columns=qdat.columns)
features_qdat = normalised_qdat[['Temperature in living room area']]
heating_target = normalised_qdat['Temperature outside the building (north side)']


#Now, we split our dataset into the training and testing dataset. Recall that we had earlier segmented the features and target variables.
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

x_train, x_test, y_train, y_test = train_test_split(features_qdat, heating_target, test_size=0.3, random_state=42)
model_qdat = LinearRegression()

#fit the model to the training dataset
model_qdat.fit(x_train, y_train)


#obtain predictions
predicted_values = model_qdat.predict(x_test)


In [3]:
# r^2
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
round(r2_score, 2)

0.64

In [4]:
#MAE
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values)
round(mae, 2)

0.08

In [5]:
rss = np.sum(np.square(y_test - predicted_values))
round(rss, 3)

66.116

In [6]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

0.106

In [7]:
model_qdat.coef_.round(2)

array([0.89])

In [8]:
# PENALISATION

#Ridge Regression
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)
ridge_pred = ridge_reg.predict(x_test)

In [9]:
rmse = np.sqrt(mean_squared_error(y_test, ridge_pred))
round(rmse, 3)

0.106

In [10]:
#Feature Selection and Lasso Regression
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)
lasso_pred = lasso_reg.predict(x_test)

In [11]:
rmse = np.sqrt(mean_squared_error(y_test, lasso_pred))
round(rmse, 3)

0.106

In [16]:
X = qdat.drop(columns=['Temperature outside the building (north side)'])
y = qdat['Appliances']

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

LinearRegression()

In [21]:
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(X_train, y_train)

Ridge(alpha=0.4)

In [22]:
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(X_train, y_train)

Lasso(alpha=0.001)

In [23]:
#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
    
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

reg_model_weights = get_weights_df(reg_model, X_train, 'Model_qdat_Weight')
ridge_weights_df = get_weights_df(ridge_reg, X_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, X_train, 'Lasso_weight')

final_weights = pd.merge(reg_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')
final_weights

,Features,Model_qdat_Weight,Ridge_Weight,Lasso_weight
0,Temperature in teenager room 2,-6.393352e-15,2.902396e-08,0.0
1,Temperature in parents room,-4.245374e-15,-6.073667e-08,0.0
2,Humidity in ironing room,-3.528722e-15,-5.123904e-09,-0.0
3,Temperature in living room area,-2.889599e-15,-3.972781e-08,0.0
4,Temperature in office room,-2.842130e-15,7.180030e-09,0.0
5,Humidity in laundry room area,-2.401981e-15,1.243607e-08,0.0
6,Temperature outside (from Chievres weather sta...,-1.763092e-15,-9.302270e-09,0.0
7,Humidity in bathroom,-3.245976e-16,7.769538e-10,0.0
8,Humidity outside (from Chievres weather station),-3.177410e-16,-1.962818e-09,-0.0
9,Windspeed,-1.971182e-16,7.064156e-09,0.0
